<h2>Initialize the workspace</h2>

In [33]:
from azureml.core import Workspace

# Load existing workspace from the config file info.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

Found the config file in: C:\Development\GitHub\marcelfranke\azureml\GetStarted\config.json
PlaygroundMLWorkspace	westeurope	Demos	westeurope


<h2>Take Image from AzureML Workspace</h2>

In [37]:
from azureml.core.image import ContainerImage

#list all images for given name
image = ContainerImage.list(ws, 'sklearn-mnist-svc')
#get the firts image from the list
image = image[0]
print(image)

ContainerImage(workspace=<azureml.core.workspace.Workspace object at 0x0000021A60CC0358>, name=sklearn-mnist-svc, id=sklearn-mnist-svc:1, tags={}, properties={}, version=1)


<h2>Create a AKS Cluster</h2>

In [12]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'aml-aks-1'
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Creating.........................................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None


<h2>Attach existing AKS cluster</h2>

In [10]:
from azureml.core.compute import AksCompute, ComputeTarget

# Get the resource id from https://porta..azure.com -> Find your resource group -> click on the Kubernetes service -> Properties
resource_id = '/subscriptions/b208dd3b-2592-4e14-a626-cd6941369193/resourcegroups/Demos/providers/Microsoft.ContainerService/managedClusters/PlaygroundAks'

# Set to the name of the cluster
cluster_name='PlaygroundAks'

# Attatch the cluster to your workgroup
aks_target = AksCompute.attach(workspace=ws, name=cluster_name, resource_id=resource_id)

# Wait for the operation to complete
aks_target.wait_for_completion(True)

Creating........
FailedProvisioning operation finished, operation "Failed"
Async operation failed with
StatusCode: 500
Message: An internal server error occurred. Please try again. If the problem persists, contact support
Compute object has provisioning state "Failed"
and provisioning errors: [{'error': {'code': 'InternalServerError', 'statusCode': 500, 'message': 'An internal server error occurred. Please try again. If the problem persists, contact support', 'details': [{'code': 'InternalError', 'message': "Retries exceeded in deploying image pull secret playgroundmlwo9484445750acrkey, exceptions occured are: Request failed with internal error. StatusCode 'InternalServerError', ReasonPhrase 'KubernetesError'. Details Operation returned an invalid status code 'Forbidden',Request failed with internal error. StatusCode 'InternalServerError', ReasonPhrase 'KubernetesError'. Details Operation returned an invalid status code 'Forbidden',Request failed with internal error. StatusCode 'Intern

<h2>Deploy your web service</h2>

In [51]:
from azureml.core.webservice import Webservice, AksWebservice

# Set configuration and service name
aks_config = AksWebservice.deploy_configuration()
aks_service_name ='sklearn-mnist-svc'

# Deploy from image
aks_service = Webservice.deploy_from_image(workspace = ws, 
                                            name = aks_service_name,
                                            image = image,
                                            deployment_config = aks_config,
                                            deployment_target = aks_target)
# Wait for the deployment to complete
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Creating service
Running..
SucceededAKS service creation operation finished, operation "Succeeded"
Healthy


<h2>Test the web service</h2>

In [48]:
import json
import numpy as np

from utils import load_data

# note we also shrink the intensity values (X) from 0-255 to 0-1. This helps the neural network converge faster

X_test = load_data('./data/test-images.gz', False) / 255.0
y_test = load_data('./data/test-labels.gz', True).reshape(-1)

# send a random row from the test set to score
random_index = np.random.randint(0, len(X_test)-1)
input_data = "{\"data\": [" + str(list(X_test[random_index])) + "]}"

headers = {'Content-Type':'application/json'}

prediction = aks_service.run(input_data = input_data)
print("POST to url", aks_service.scoring_uri)
#print("input data:", input_data)
print("label:", y_test[random_index])
print("prediction:", prediction)

POST to url http://40.68.199.133/api/v1/service/aml-aks-10cf707430ab/score
label: 6
prediction: [6]


<h2>Cleanup</h2>

In [52]:
aks_service.delete()
aks_target.delete()